# Day Eleven

## Task

The [task_](https://adventofcode.com/2017/day/11) is to implement some sort of path-finding in a hex grid.

### Part One

In the first part, we are given a list of movement instructions. Our goal is to find the distance from the origin to the point that one reaches after following these instructions.

I will use the fact that certain movements can be reduced to other movements. For example, moving `ne` and `nw` is equivalent to moving `n`. Similarly, moving `ne` and `sw` is equivalent to not moving at all.

In [1]:
from collections import Counter

directions = Counter(['ne', 'ne', 's', 's'])
directions

Counter({'ne': 2, 's': 2})

Now, I know that `ne` and `s` is the same as moving `se`. Therefore,

In [2]:
n = min(directions['ne'], directions['s'])

directions['se'] = n
directions['ne'] -= n
directions['s'] -= n

directions

Counter({'ne': 0, 's': 0, 'se': 2})

How do we know when a counter of directions can be reduced no further? Let's play with some more examples to see if a pattern reveals itself.

In [3]:
directions = Counter(['ne', 'ne', 'sw', 'sw', 'ne', 'ne', 's', 's', 'se', 'sw', 'se', 'sw', 'sw'])
directions

Counter({'ne': 4, 's': 2, 'se': 2, 'sw': 5})

It's clear that those with opposite vertical directions (e.g. `ne` and `s`, `se` and `n`, ...) can be reduced. Further, those with the same vertical direction but opposite horizontal directions can be too. Finally, opposing directions equate to no movement at all. With this in mind, let's define the pairs that can be reduced and what they can be reduced to.

In [4]:
pairs = {
    (('ne', 'nw'), 'n'), 
    (('ne', 'sw'), ''),
    (('ne', 's'), 'se'),
    (('nw', 'ne'), 'n'), 
    (('nw', 'se'), ''),
    (('nw', 's'), 'sw'),
    (('se', 'sw'), 's'), 
    (('se', 'nw'), ''),
    (('se', 'n'), 'ne'),
    (('sw', 'se'), 's'), 
    (('sw', 'ne'), ''),
    (('sw', 'n'), 'nw'),
    (('n', 'se'), 'ne'),
    (('n', 'sw'), 'nw'),
    (('n', 's'), ''),
    (('s', 'ne'), 'se'),
    (('s', 'nw'), 'sw'),
    (('s', 'n'), ''),
}

With this list, we can search our counter to find reducible pairs.

In [5]:
def reduce(directions, first_direction, second_direction, composed_direction):
    n = min(directions[first_direction], directions[second_direction])
    
    directions[first_direction] -= n
    directions[second_direction] -= n    
    if composed_direction:
        directions[composed_direction] += n
    
    return bool(n)


def iterate_until(directions, pairs):
    while True:
        for pair in pairs:
            if reduce(directions, pair[0][0], pair[0][1], pair[1]):
                break
        else:
            return directions

        
directions = Counter(['ne', 'ne', 'sw', 'sw', 'ne', 'ne', 's', 's', 'se', 'sw', 'se', 'sw', 'sw'])
iterate_until(directions, pairs)

Counter({'n': 0, 'ne': 0, 'nw': 0, 's': 3, 'se': 1, 'sw': 0})

The shortest path is then given by summing the values. Let's test this on the given examples.

In [6]:
assert 3 == sum(iterate_until(Counter(['ne', 'ne', 'ne']), pairs).values())
assert 0 == sum(iterate_until(Counter(['ne', 'ne', 'sw', 'sw']), pairs).values())
assert 2 == sum(iterate_until(Counter(['ne', 'ne', 's', 's']), pairs).values())
assert 3 == sum(iterate_until(Counter(['se', 'sw', 'se', 'sw', 'sw']), pairs).values())

Finally, let's do this on the example.

In [7]:
with open('../../data/day11.txt') as f:
    data = f.read()
    
sum(iterate_until(Counter(data.strip().split(',')), pairs).values())

720

### Part Two

For part two, I need to work out the furthest distance away from the origin at any point along the path. That is, I need to calculate this value for every subsequence of instruction.

For example, for instruction set `se,se,n,n`, I would need to work out the values for `se`, `se,se`, `se,se,n` and `se,se,n,n` and work out the maximum distance across these.

In [8]:
with open('../../data/day11.txt') as f:
    data = f.read()
    
directions = data.strip().split(',')
all_directions = [Counter(directions[0:i]) for i in range(1, len(directions) + 1)]
max(map(lambda x: sum(iterate_until(x, pairs).values()), all_directions) )

1485